In [1]:
import pandas as pd
import torch
import PIL.Image
from torchvision import transforms
import  numpy as np
import  torch
import torchvision
import torch.nn.functional as F
from torch import  nn
from torch import optim
import math

# 数据集预处理

In [ ]:
data_file = "data/01_LungPredictData05.csv"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 100)

## 处理数值类型

In [ ]:
data = pd.read_csv(data_file,sep=',')
data

In [ ]:


all_features = data.iloc[:,:15]
outputs  = data.iloc[:,15]

strlist = ['sex','inType','outType','recover','fever','pastScore','right_up','right_mid','right_down',
           'left_up','left_down']
all_features[strlist] = all_features[strlist].astype(np.str)

print(all_features.dtypes)

In [ ]:
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index

# print(numeric_features[1])
# print(all_features[numeric_features])

all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features = pd.get_dummies(all_features, dummy_na=False)
all_features
data = pd.concat([all_features,outputs],axis=1)
data


    ## 数据集输入为inputs 输出为outpus 此时均为Dataframe类型

In [ ]:
import  openpyxl
# xlsx_outputpath='data/after_lungdataset.xlsx'
# csv_outputpath='data/after_lungdataset.csv'
# all_features.to_excel(xlsx_outputpath,index=True,header=True)
# all_features.to_csv(csv_outputpath,sep=',',index=False,header=False)
# inputs = all_features
# outputs = outputs
# data = pd.concat([inputs, outputs], sort=False, axis=1)
#
# data = pd.read_csv('data/Data.csv',sep=';')
#
# # 住院时长周数为值
# y = data.iloc[:, 48]
#
# # 定义特征
# x = data.iloc[:, 1:48]


# 划分数据集

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(
    all_features,outputs, test_size=0.2, random_state=None)
X_train = x_train
X_test = x_test
x_train

## 计算各类总数

In [ ]:
# # 训练集各类总数
# train_class_num_list = [0,0,0]
# # double_list = outputs
# for id,value in enumerate(y_train):
#     if value == 0:
#         train_class_num_list[0] = train_class_num_list[0] + 1
#     elif value == 1:
#         train_class_num_list[1] = train_class_num_list[1] + 1
#     else:
#         train_class_num_list[2] = train_class_num_list[2] + 1
#
# # train_np_class_num_list = np.array([train_class_num_list[0],train_class_num_list[1],train_class_num_list[2]])
# train_tensor_class_num_list = torch.tensor([train_class_num_list[0],train_class_num_list[1],train_class_num_list[2]])
#
#
# # 测试集各类总数
# test_class_num_list = [0,0,0]
# # double_list = outputs
# for id,value in enumerate(y_test):
#     if value == 0:
#         test_class_num_list[0] = test_class_num_list[0] + 1
#     elif value == 1:
#         test_class_num_list[1] = test_class_num_list[1] + 1
#     else:
#         test_class_num_list[2] = test_class_num_list[2] + 1
#
#
# # train_np_class_num_list = np.array([test_class_num_list[0],test_class_num_list[1],test_class_num_list[2]])
# test_tensor_class_num_list = torch.tensor([test_class_num_list[0],test_class_num_list[1],test_class_num_list[2]])
#
# train_tensor_class_num_list,test_tensor_class_num_list

In [ ]:
#输入
x_train = torch.tensor(x_train.values)
x_test = torch.tensor(x_test.values)
#输出
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

data_dim = x_train.shape[1]
# target_dim = target.shape[1]
# print(data)
print(data.shape)
print(data_dim,x_train.shape)
x_train.unique()

## 定义Dataset

In [ ]:
from torch.utils.data import Dataset


class LungDataset(Dataset):

    def __init__(self,inputs,outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __getitem__(self, idx):
        vector = self.inputs[idx]
        label = self.outputs[idx]
        return vector,label

    def __len__(self):
        return len(self.outputs)

trainDataset = LungDataset(x_train,y_train)
testDataset = LungDataset(x_test,y_test)
x,y = trainDataset[0]
x.shape



## 随机种子


In [ ]:
# from torch.backends import cudnn
# import random
# cudnn.benchmark = False            # if benchmark=True, deterministic will be False
# cudnn.deterministic = True
#
# GLOBAL_SEED = 1
#
# def set_seed(seed):
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
#
# set_seed(1)
#
# GLOBAL_WORKER_ID = None
# def worker_init_fn(worker_id):
#     global GLOBAL_WORKER_ID
#     GLOBAL_WORKER_ID = worker_id
#     set_seed(GLOBAL_SEED + worker_id)
# # dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=2, worker_init_fn=worker_init_fn)

# 搭建网络

In [ ]:
input_dim = x_train.shape[1]
hidden_dim = 10
class MnistNet(nn.Module):
    def __init__(self):
        super(MnistNet,self).__init__()
        self.fc1 = nn.Linear(input_dim,hidden_dim)  #定义Linear的输入和输出的形状
        self.fc2 = nn.Linear(hidden_dim,3)  #定义Linear的输入和输出的形状

    def forward(self,x):
        x = x.view(-1,input_dim)  #对数据形状变形，-1表示该位置根据后面的形状自动调整
        x = self.fc1(x) #[batch_size,28]

        # x = F.relu(x)  #[batch_size,28]
        # x = self.fc1_1(x) #[batch_size,10]

        x = F.relu(x)  #[batch_size,28]
        x = self.fc2(x) #[batch_size,10]
        return F.log_softmax(x, dim=-1)

In [ ]:

# train_dataloader = torch.utils.data.DataLoader(data,batch_size=64,shuffle=True)

### 损失函数

In [ ]:
import torch
from torch.nn.modules.loss import _Loss
import torch.nn.functional as F
import json


class BalancedSoftmax(_Loss):
    """
    Balanced Softmax Loss
    """
    def __init__(self, freq_path):
        super(BalancedSoftmax, self).__init__()
        with open(freq_path, 'r') as fd:
            freq = json.load(fd)
        freq = torch.tensor(freq)
        self.sample_per_class = freq

    def forward(self, input, label, reduction='mean'):
        return balanced_softmax_loss(label, input, self.sample_per_class, reduction)


def balanced_softmax_loss(labels, logits, sample_per_class, reduction):
    """Compute the Balanced Softmax Loss between `logits` and the ground truth `labels`.
    Args:
      labels: A int tensor of size [batch].
      logits: A float tensor of size [batch, no_of_classes].
      sample_per_class: A int tensor of size [no of classes].
      reduction: string. One of "none", "mean", "sum"
    Returns:
      loss: A float tensor. Balanced Softmax Loss.
    """
    spc = sample_per_class.type_as(logits)

    spc = spc.unsqueeze(0).expand(logits.shape[0], -1)

    logits = logits + spc.log()

    loss = F.cross_entropy(input=logits, target=labels, reduction=reduction)
    return loss


def create_loss(freq_path):
    print('Loading Balanced Softmax Loss.')
    return BalancedSoftmax(freq_path)

# 准备迭代器

In [ ]:
train_batch_size = 64
test_batch_size = 128
img_size = data_dim

def get_dataloader(train=True):

    #准备数据集，其中0.1307，0.3081为MNIST数据的均值和标准差，这样操作能够对其进行标准化
    #因为MNIST只有一个通道（黑白图片）,所以元组中只有一个值
    dataset = trainDataset if train else testDataset
    #准备数据迭代器
    batch_size = train_batch_size if train else test_batch_size
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)
    return dataloader

# 训练

In [ ]:
mnist_net = MnistNet().double()
optimizer = optim.Adam(mnist_net.parameters(),lr= 0.001)
train_loss_list = []
train_count_list = []
def train(epoch):
    mnist_net.train(True)
    train_dataloader = get_dataloader(True)
    optimizer.zero_grad()
    print("开始训练：")
    for idx,(data,target) in enumerate(train_dataloader):
        output = mnist_net(data)
        # loss = F.cross_entropy(output,target) #对数似然损失
        loss = F.nll_loss(output,target) #对数似然损失
        # loss = balanced_softmax_loss(labels = target, logits=output, reduction='mean', sample_per_class=train_tensor_class_num_list)
        loss.backward()
        optimizer.step()
        if idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,  idx * len(data), len(train_dataloader.dataset),100. * idx / len(train_dataloader), loss.item()))
            train_loss_list.append(loss.item())
            train_count_list.append(idx*train_batch_size+(epoch-1)*len(train_dataloader))
    print("结束训练。")

print(mnist_net)

In [ ]:
def acc_train():
    test_loss = 0
    correct = 0
    mnist_net.eval()
    test_dataloader = get_dataloader(train=True)
    with torch.no_grad():
        for data, target in test_dataloader:
            output = mnist_net(data)
            test_loss += F.nll_loss(output, target, reduction='mean').item()
            pred = output.data.max(1, keepdim=True)[1] #获取最大值的位置,[batch_size,1]

            # test_loss += balanced_softmax_loss(labels = target,
            #                                    logits=(output), reduction='mean',
            #                                    sample_per_class=train_tensor_class_num_list).item()
            # #output-math.log（）
            # spc = test_tensor_class_num_list.type_as(output)
            # spc = spc.unsqueeze(0).expand(output.shape[0], -1)
            # output = output - spc.log()
            # pred = output.data.max(1, keepdim=True)[1] #获取最大值的位置,[batch_size,1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_dataloader.dataset)

    print('\nTrain set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_dataloader.dataset),
        100. * correct / len(test_dataloader.dataset)))
    correct_rate = 100. * correct / len(test_dataloader.dataset)
    return correct_rate

# 测试

In [ ]:
def test():
    test_loss = 0
    correct = 0
    mnist_net.eval()
    test_dataloader = get_dataloader(train=False)
    with torch.no_grad():
        for data, target in test_dataloader:
            output = mnist_net(data)
            test_loss += F.nll_loss(output, target, reduction='mean').item()

            # test_loss += balanced_softmax_loss(labels = target,
            #                                    logits=(output), reduction='mean',
            #                                    sample_per_class=test_tensor_class_num_list).item()
            #output-math.log（）
            # spc = test_tensor_class_num_list.type_as(output)
            # spc = spc.unsqueeze(0).expand(output.shape[0], -1)
            # output = output - spc.log()

            pred = output.data.max(1, keepdim=True)[1] #获取最大值的位置,[batch_size,1]
            correct += pred.eq(target.data.view_as(pred)).sum()


    test_loss /= len(test_dataloader.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_dataloader.dataset),
        100. * correct / len(test_dataloader.dataset)))
    test_correct = 100. * correct / len(test_dataloader.dataset)
    return test_correct,test_loss

In [ ]:
from matplotlib import pyplot as plt
import PIL
epoch = 400
#定义两个数组
Loss_list = []
Accuracy_list = []
test_Loss_list = []
test_Accuracy_list = []
# 训练
for i in range(epoch):
    train(i)
    correct_rate = acc_train()
    # Loss_list.append(c)
    Accuracy_list.append(correct_rate)

    test_correct,test_loss = test()
    test_Accuracy_list.append(test_correct)
    test_Loss_list.append(test_loss)





Loss_list = train_loss_list

#我这里迭代了200次，所以x的取值范围为(0，200)，然后再将每次相对应的准确率以及损失率附在x上
x1 = range(0, epoch)
x2 = range(0, epoch)
y1 = Accuracy_list
y2 = Loss_list
plt.subplot(2, 1, 1)
plt.plot(x1, y1, 'o-')
plt.title('Train accuracy vs. epoches')
plt.ylabel('Train accuracy')
plt.subplot(2, 1, 2)
plt.plot(x2, y2, '.-')
plt.xlabel('Train loss vs. epoches')
plt.ylabel('Train loss')
plt.savefig("accuracy_loss.jpg")
plt.show()

x1 = range(0, epoch)
x2 = range(0, epoch)
y1 = test_Accuracy_list
y2 = test_Loss_list
plt.subplot(2, 1, 1)
plt.plot(x1, y1, 'o-')
plt.title('test accuracy vs. epoches')
plt.ylabel('test accuracy')
plt.subplot(2, 1, 2)
plt.plot(x2, y2, '.-')
plt.xlabel('test loss vs. epoches')
plt.ylabel('test loss')
plt.savefig("test_accuracy_loss.jpg")
plt.show()


In [ ]:
test()

In [ ]:
import shap

explainer = shap.DeepExplainer(mnist_net, torch.from_numpy(X_train.values).double())
shap_values = explainer.shap_values(torch.from_numpy(X_test.values).double())
shap.summary_plot(shap_values, X_test, plot_type='bar')
class_names=["1-3 week","3-6week","6+week"]
# shap.summary_plot(shap_values[2], X_test, class_names=class_names)
# shap.plots.heatmap(shap_values)


In [ ]:
torch.save(mnist_net.state_dict(),"../mnist_net.pt") #保存模型参数
torch.save(optimizer.state_dict(), '../mnist_optimizer.pt') #保存优化器参数

In [ ]:
mnist_net.load_state_dict(torch.load("../mnist_net.pt"))
optimizer.load_state_dict(torch.load("../mnist_optimizer.pt"))